## FBA simulation toy_atp
Simulating the FBA part.

In [1]:
from __future__ import print_function, absolute_import

import pandas as pd
import cobra
from matplotlib import pyplot as plt

from sbmlutils import fbc
from sbmlutils.dfba.analysis import set_matplotlib_parameters
from sbmlutils.dfba.toy_atp import model_factory

set_matplotlib_parameters()

In [4]:
# load model
sbml_path = './results/v{}/toy_atp_fba.xml'.format(model_factory.version)
print(sbml_path)
model = cobra.io.read_sbml_model(sbml_path)
cobra.io.sbml3.validate_sbml_model(sbml_path)

./results/v1/toy_atp_fba.xml


(<Model toy_atp_fba at 0x7f69357a7b00>,
 {'SBML errors': [], 'other': [], 'validator': [], 'warnings': []})

In [7]:
# objective function & boundaries
# pprint(mfba.objective)
df = fbc.cobra_reaction_info(model)
print(df)
print("reactions:", len(model.reactions))
print("metabolites:", len(model.metabolites))
print("genes:", len(model.genes))

          lb     ub reversibility boundary objective_coefficient  \
R1         0   1000         False    False                     0   
R2         0   1000         False    False                     0   
R3         0   1000         False    False                     0   
RATP       0  0.001         False    False                     1   
EX_atp -1000   1000          True     True                     0   
EX_adp -1000   1000          True     True                     0   
EX_glc -1000   1000          True     True                     0   
EX_pyr -1000   1000          True     True                     0   

             forward_variable                     reverse_variable  
R1        0.0 <= R1 <= 1000.0           0 <= R1_reverse_cda52 <= 0  
R2        0.0 <= R2 <= 1000.0           0 <= R2_reverse_8c6d2 <= 0  
R3        0.0 <= R3 <= 1000.0           0 <= R3_reverse_5c108 <= 0  
RATP     0.0 <= RATP <= 0.001         0 <= RATP_reverse_50e25 <= 0  
EX_atp  0 <= EX_atp <= 1000.0  0 <= EX_atp

In [8]:
# Exchange reactions
ex_idx = df.index.str.contains('^EX_')
df[ex_idx]

,lb,ub,reversibility,boundary,objective_coefficient,forward_variable,reverse_variable
EX_atp,-1000,1000,True,True,0,0 <= EX_atp <= 1000.0,0 <= EX_atp_reverse_dfd97 <= 1000.0
EX_adp,-1000,1000,True,True,0,0 <= EX_adp <= 1000.0,0 <= EX_adp_reverse_76b83 <= 1000.0
EX_glc,-1000,1000,True,True,0,0 <= EX_glc <= 1000.0,0 <= EX_glc_reverse_8e37d <= 1000.0
EX_pyr,-1000,1000,True,True,0,0 <= EX_pyr <= 1000.0,0 <= EX_pyr_reverse_03be5 <= 1000.0


In [9]:
# optimize
s = model.optimize(objective_sense="maximize")
model.summary(fva=True)

IN FLUXES                     OUT FLUXES                    OBJECTIVES
----------------------------  ----------------------------  ------------
id      Flux  Range           id      Flux  Range           RATP  0.001
----  ------  --------------  ----  ------  --------------
atp    0.001  [0.001, 0.001]  adp    0.001  [0.001, 0.001]
glc    0      [0, 500]        pyr    0      [0, 1e+03]


In [ ]:
# pfba (minimal flux)
# no difference, the flux variability analysis 
# already showed us that the model has unique solution under given bounds 
s = model.optimize(objective_sense="maximize")
cobra.flux_analysis.pfba(model)
model.summary(fva=True)

Network map:  
https://escher.github.io/builder/index.html?enable_editing=true&map_name=e_coli_core.Core%20metabolism

In [ ]:
# what happens to glucose ?
# single reaction to 
print(model.metabolites.glc__D_e.summary())

In [ ]:
# What is going on with the ATPM reaction
print(model.reactions.ATPM.reaction)

In [ ]:
# gene and reaction deletions
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
# single_gene_deletion(model)
import numpy as np
r_del = single_reaction_deletion(model)
r_del['percent'] = np.round(r_del['flux']/0.9166, 3)
r_del.sort_values(by=['flux'])

## Change the bounds
Driving the model via changing the bounds of the exchange reactions
* in coupling based on maximal uptake rates and availability of substrate

In [ ]:
import numpy as np
import pandas as pd

# set of bounds (we can't go to zero, a minimal amount of glc is needed for biomass generation)
# More complete models could switch to alternative carbon sources
glc_bounds = np.linspace(-10, -0.5, 10)

results = []
for lb_glc in glc_bounds:
    # set the lower bound (uptake direction) of the exchange reaction
    print(lb_glc)
    model.reactions.get_by_id("EX_glc__D_e").lower_bound = lb_glc
    # pFBA
    s = model.optimize(objective_sense="maximize")
    s = cobra.flux_analysis.pfba(model)
    model.summary(fva=True)
    
    fluxes = s.fluxes
    results.append(fluxes)
df = pd.DataFrame(results)

In [ ]:
# Find Exchange reactions & Internal Reactions
ex_rids = []
in_rids = []
for r in model.reactions:
    rid = r.id
    if (rid.startswith('EX_')):
        ex_rids.append(rid)
    else:
        in_rids.append(rid)
        
# print(ex_rids)
# print()
# print(in_rids)

In [ ]:
# Plot internal, external, and biomass reactions
fig, ((ax1, ax2),(ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))

for rid in ex_rids:
    ax1.plot(glc_bounds, df[rid], marker='s', alpha=0.8, label=rid)

for rid in in_rids:
    ax2.plot(glc_bounds, df[rid], marker='s', alpha=0.8, label=rid)

ax3.plot(glc_bounds, df["BIOMASS_Ecoli_core_w_GAM"], color="black", label="biomass",
        marker="s")
    
for ax in [ax1, ax2, ax3]:
    ax.set_xlabel('lb glc (EC_glc__D_e)')
    ax.set_xlim(-10,3)

ax1.legend()
ax3.legend()

plt.show()
fig.savefig("ecoli_glc_dependency.png", bbox="tight")